# Dash Interface

## Importing modules 

In order to use this file some python modules need to be installed. You can use the .yml file in the repository to do so

In [1]:
from os.path import dirname, join, isfile
from itertools import chain
import pandas as pd
import numpy as np
import scipy.signal
import scipy.io as sio
import matplotlib.pyplot as plt
from jupyter_dash import JupyterDash  # pip install dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input
import pandas as pd  # pip install pandas
import plotly.express as px
import math
from dash import no_update
from dash_canvas import DashCanvas

## Function to get dataframe

In [2]:
def get_dataframe(mat_fpath):
    ''' Obtain Pandas dataframe from mat file
    :param mat_fpath: File path to the mat file from which pandas DF should be created
    '''
    assert isfile(mat_fpath), "Invalid file path!"
    mat_contents = sio.loadmat(mat_fpath)
    viz_data = mat_contents["ccFddData"]
    viz_data = mat_contents['ccFddData']
    n = 5;
    iFDD_names = ["Sys2_Impact_Location_P{}".format(str(i+1)) for i in range(n)]
    dFDD_pnames = ["Sys2_Dmg_P_{}".format(str(i+1)) for i in range(n)]
    dFDD_cpnames = ["Sys2_Dmg_P_{}_c".format(str(i+1)) for i in range(n)]
    dFDD_names = list(chain.from_iterable(zip(dFDD_pnames, dFDD_cpnames)))
    col_names = ['Time', 'Sys2_Total_Dmg', 'Sys2_SimFDD_xe'] + iFDD_names + dFDD_names
    df = pd.DataFrame(viz_data, columns = col_names)
    return df             

In [3]:
data = get_dataframe("newData_v6.mat")

In [4]:
t = data['Time'].to_numpy()
trueHS = data['Sys2_Total_Dmg'].to_numpy()
sim_fdd_xe = data['Sys2_SimFDD_xe'].to_numpy()
impact_fdd = np.vstack(([data['Sys2_Impact_Location_P{}'.format(ii+1)].to_numpy() for ii in range(5)])).T
dmg_fdd = [np.vstack((data['Sys2_Dmg_P_{}'.format(ii+1)].to_numpy(), \
                     data['Sys2_Dmg_P_{}_c'.format(ii+1)].to_numpy())).T for ii in range(5)]

## Function to graph FDD data

In [5]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
#statically graphs FDD data in a subplot
def FDD_graphs(st, ft):
  fig = make_subplots(rows=3, cols=2, 
                      subplot_titles=("Damage FDD: Segment 1", "Damage FDD: Segment 2", 
                      "Damage FDD: Segment 3", "Damage FDD: Segment 4", 
                      "Damage FDD: Segment 5"))


  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[0][st*1000:ft*1000,0], name="P[Damage] seg 1"),
      row = 1, col = 1
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[0][st*1000:ft*1000,1], name="P[No Damage] seg 1"),
      row = 1, col = 1
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[1][st*1000:ft*1000,0], name="P[Damage] seg 2"),
      row = 1, col = 2
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[1][st*1000:ft*1000,1], name="P[No Damage] seg 2"),
      row = 1, col = 2
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[2][st*1000:ft*1000,0], name="P[Damage] seg 3"),
      row = 2, col = 1
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[2][st*1000:ft*1000,1], name="P[No Damage] seg 3"),
      row = 2, col = 1
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[3][st*1000:ft*1000,0], name="P[Damage] seg 4"),
      row = 2, col = 2
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[3][st*1000:ft*1000,1], name="P[No Damage] seg 4"),
      row = 2, col = 2
   )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[4][st*1000:ft*1000,0], name="P[Damage] seg 5"),
      row = 3, col = 1
  )
  fig.add_trace(
      go.Scatter(x = t, y=dmg_fdd[4][st*1000:ft*1000,1], name="P[No Damage] seg 5"),
      row = 3, col =1
  )


  #fig.update_layout(height=1000, width=900, title_text="FDD data for damage in each segment")
  return fig

## Interface to show data

In [7]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app_testtt = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app_testtt.layout = html.Div([
    dcc.Dropdown( ##Dropdown that gives option for user to select a subsystem
        id="sub_to_choose", ##Element id, used to refer to it
        options=[ ##dictionary with label of each option, alongside the value if returns to the code
            {'label': 'Structure Sys', 'value': 'struc'},
            {'label': 'Power Sys', 'value': 'pow'},
            {'label': 'Blank sys', 'value': 'blank'}
        ],
        value='struc'
    ),
    dcc.Graph( ##Graphing element, allows plotly graphs to be displayed
        id="updating_graph"
    ),
    
    dcc.Interval( ##Interval element, counts how many intervals have passed
        id="count_sec",
        interval=5*1000, # in milliseconds, amount of time it takes to update n_intervals
        n_intervals=0 #how many intervals have passed
    )

])
##The callback is how the app gets updated. Every time one of the inputs change, the function belows it is executed
@app_testtt.callback(Output('updating_graph', 'figure'), #Output is whatever the function is returning. In this case,
                        #it is reurning the figure of the dcc.graph element
              [Input("sub_to_choose", "value"),Input('count_sec', 'n_intervals')]) #Inputs are the inputs of the function below
def update_metrics(opt, n): 
    if opt == "struc":
        fig = FDD_graphs(0, 5*n)
    return fig


app_testtt.run_server(debug=False, port=8020)
#After this cell is executed a link will be generated below. If you copy the link and paste it on your browser 
#You should be able to see the app

C:\Users\Bernardo Alvarenga\anaconda3\envs\rethi\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

127.0.0.1 - - [10/Aug/2021 20:41:15] "GET /_shutdown_e7d9750c-6d73-44ab-b47e-ddaae7ccb77f HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8020/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Aug/2021 20:41:15] "GET /_alive_e7d9750c-6d73-44ab-b47e-ddaae7ccb77f HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8020/
